1) Questão 1: Implemente uma estratégia de trade automático baseado em duas médias móveis e verifique o desempenho quanto ao Índice Sharpe. Utilize um período mínimo de 3 anos e os 5 (cinco) ativos escolhidos na primeira avaliação.

In [1]:
import datetime
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

acoes = ['CMIG4','ELET3','ENBR3','USIM5','GOAU4']


def download_acao(ticker,data_inicio,data_fim):
    yf.pdr_override()       
    ticker = ticker + '.SA'
    return pdr.get_data_yahoo(ticker, 
                          start=data_inicio, 
                          end=data_fim)
CMIG4_df = pd.DataFrame()
ELET3_df = pd.DataFrame()
ENBR3_df = pd.DataFrame()
USIM5_df = pd.DataFrame()
GOAU4_df = pd.DataFrame()


In [2]:
CMIG4_df = download_acao('CMIG4',datetime.datetime(2018,9,1),datetime.datetime(2019,9,1))

KeyboardInterrupt: 